In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from sympy.physics.units import momentum
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

### 记载历史数据文件

In [53]:
dataset = pd.read_csv('./data/load.csv', index_col=[0])
dataset = dataset.fillna(method='pad')
dataset = np.array(dataset)
# 将所有的数据放到一个列表里面，方便后续的训练集和测试集的制作
a = []
for item in dataset: # item为一整天的数据
    for i in item:
        a.append(i)
frame = pd.DataFrame(a)
frame

,0
0,5687.054687
1,5460.153257
2,5288.463742
3,5078.027286
4,4918.009490
...,...
35035,6798.593207
35036,6553.228700
35037,6321.855764
35038,6151.673338


### 划分训练集和验证集

In [54]:
train = frame.iloc[0:int(len(a)*0.8), [0]] # 序列数据，只能按顺序划分数据集，前80%作为训练集，80%~90%作为验证集
val = frame.iloc[int(len(a)*0.8):int(len(a)*0.9), [0]] # 百分之80至90之间的数据作为验证集
print(len(train),len(val))

28032 3504


### 进行数据归一化，将数据归一化到0-1之间

In [55]:
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train)
val = scaler.fit_transform(val)

### 设置训练集的特征列表和对应标签列表

In [56]:
x_train = []
y_train = []
for i in range(96,len(train)):
    x_train.append(train[i-96:i]) # 前面96个数据作为特征
    y_train.append(train[i])

### 设置验证集的特征列表和对应的标签列表

In [57]:
x_val = []
y_val = []
for i in range(96,len(val)):
    x_val.append(train[i-96:i]) # 前面96个数据作为特征
    y_val.append(train[i])

In [58]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_val,y_val=np.array(x_val),np.array(y_val)
print(x_train.shape,y_train.shape) # x_train的形状为(样本数,时间步,特征数)
print(x_val.shape,y_val.shape)

(27936, 96, 1) (27936, 1)
(3408, 96, 1) (3408, 1)


In [59]:
class  LOAD_MODEL(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(LOAD_MODEL, self).__init__()
        # 第一层LSTM
        self.lstm1 = nn.LSTM(input_size=input_size,
                             hidden_size=hidden_size,
                             num_layers=1,# 层数默认为1
                             batch_first=True, # 输入/输出的形状为(批量,时间步,特征数)
                             nonlinearity='relu', # 机激活函数为relu
                             return_sequences=True # 保留所有时间步输出
                             )
        # 第二层lstm层
        self.lstm2=nn.LSTM(input_size=hidden_size, # 第2层输入数等于第1层的输出数
                           hidden_size=hidden_size,
                           num_layers=1,
                           batch_first=True,
                           nonlinearity='relu', # ptorch中lstm默认的激活函数为tanh,需要手动指定为relu
                           return_sequences=False # 仅保留最后一个时间步输出
                            )
        # 第一层全连接层
        self.fc1 = nn.Linear(hidden_size, 10)
        # 第二层全连接层
        self.fc2 = nn.Linear(10, 1)
        # relu激活函数
        self.relu = nn.ReLU()
    def forward(self, x):
        # 经过第一个lstm层,保留所有的时间步
        out, (h, c) = self.lstm1(x) # 输出形状为(批量数,步长,隐藏单元数)
        # 经过第二个lstm层
        out, (h, c) = self.lstm2(out) # 输出形状为(批量数,步长,隐藏单元数)
        out=out[:,-1,:] # 取最后一个时间步,形状为(批量数,隐藏单元数)
        # 经过第一个全连接层
        out = self.fc1(out)
        out = self.relu(out)
        # 经过第二个全连接层
        out = self.fc2(out)
        return out



### 将numpy数组转化为pytorch张量

In [60]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

### 封装为TensorDataset（直接绑定输入和标签，适合简单场景）

In [63]:
train_dataset = TensorDataset(x_train_tensor, y_train_tensor) # 包含了特征和标签
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

### 创建dataloader加载器

In [65]:
batch_size = 64  # 根据GPU显存调整（如16/32/64）
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=False,  # 时序数据不能打乱
    num_workers=0   # 多线程加载（Windows建议设0，Linux/Mac可设2/4）
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=False,  # 验证集不打乱，保证评估稳定
    num_workers=0
)

In [ ]:
# 训练时的标签维度调整示例（在训练循环中）
model = LOAD_MODEL(10,10)
criterion=nn.MSELoss()
for batch_x, batch_y in train_loader:
    batch_y = batch_y.unsqueeze(1)  # 从(N,) → (N,1)
    outputs = model(batch_x)
    loss = criterion(outputs, batch_y)